In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
!pip install kaggle

In [2]:
creds = '{}'

In [3]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
path = Path('us-patent-phrase-to-phrase-matching')

In [5]:
if not iskaggle and not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|██████████| 682k/682k [00:00<00:00, 46.0MB/s]

In [6]:
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets

In [7]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [8]:
import pandas as pd

In [9]:
df = pd.read_csv(path/'train.csv')
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [10]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [11]:
df['input'] = 'TEXT1:' + df.context + '; TEXT2:' + df.target + '; ANC1:' + df.anchor
df.input.head()

0    TEXT1:A47; TEXT2:abatement of pollution; ANC1:...
1      TEXT1:A47; TEXT2:act of abating; ANC1:abatement
2     TEXT1:A47; TEXT2:active catalyst; ANC1:abatement
3    TEXT1:A47; TEXT2:eliminating process; ANC1:aba...
4       TEXT1:A47; TEXT2:forest region; ANC1:abatement
Name: input, dtype: object

In [12]:
df.head()

,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1:A47; TEXT2:abatement of pollution; ANC1:...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1:A47; TEXT2:act of abating; ANC1:abatement
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1:A47; TEXT2:active catalyst; ANC1:abatement
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1:A47; TEXT2:eliminating process; ANC1:aba...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1:A47; TEXT2:forest region; ANC1:abatement


In [ ]:
!pip install datasets

In [14]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

# **now we will start tokenisation and numericalisation**

AutoTokenizer will create a tokenizer appropriate for a given model

In [ ]:
pip install --upgrade pip

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "microsoft/deberta-v3-small" # you can specify the model size here
tokz = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [19]:
def tok_func(x): return tokz(x["input"])
# a simple function which tokenizes our inputs

In [ ]:
tok_ds = ds.map(tok_func, batched=True)

In [21]:
row = tok_ds[0]
row

{'id': '37d61fd2272659b1',
 'anchor': 'abatement',
 'target': 'abatement of pollution',
 'context': 'A47',
 'score': 0.5,
 'input': 'TEXT1:A47; TEXT2:abatement of pollution; ANC1:abatement',
 'input_ids': [1,
  54453,
  435,
  294,
  558,
  5753,
  346,
  54453,
  445,
  294,
  16191,
  297,
  17019,
  265,
  6435,
  346,
  23702,
  435,
  294,
  16191,
  297,
  17019,
  2],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [22]:
row['input'], row['input_ids']

('TEXT1:A47; TEXT2:abatement of pollution; ANC1:abatement',
 [1,
  54453,
  435,
  294,
  558,
  5753,
  346,
  54453,
  445,
  294,
  16191,
  297,
  17019,
  265,
  6435,
  346,
  23702,
  435,
  294,
  16191,
  297,
  17019,
  2])

 *we need to prepare our labels, transformers always assumes that labels have the column name label, but in our dataset its currently aka score*

In [23]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

# now that weve prepared our tokens and labels, creating val set->

In [24]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [25]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds # datasetdict

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [ ]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)